In [1]:
import pandas as pd
import numpy as np

data_raw = pd.read_csv('../raw/지역사회건강조사 1차 전처리.csv')

In [2]:
data_raw

,Unnamed: 0,만나이,성별,주관적 건강수준,격렬한 신체활동 일수,격렬한 신체활동 시간(시),격렬한 신체활동 시간(분),중등도 신체활동 일수,중등도 신체활동 시간(시),중등도 신체활동 시간(분),걷기 실천 일수,걷기 실천 시간(시),걷기 실천 시간(분),체중조절 경험 여부,주관적 스트레스 수준,우울감 경험 여부,행복감 지수,사회환경_동네 자연환경,사회환경_동네 생활환경,자치구
0,0,61,2,4,1,5,0,3,1,0,7,1,0,1,3,2,5,2,1,마포구
1,1,66,1,3,1,5,0,0,88,88,7,1,0,4,2,2,8,2,1,마포구
2,2,89,2,5,0,88,88,0,88,88,0,88,88,4,1,2,77,9,9,마포구
3,3,27,1,4,1,0,10,1,0,10,7,0,30,3,2,2,8,1,1,마포구
4,4,42,2,3,2,0,30,2,0,10,5,0,20,1,3,2,6,2,2,마포구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22925,22925,62,2,3,0,88,88,7,1,0,5,0,20,1,2,2,4,9,1,금천구
22926,22926,65,1,4,7,2,0,7,2,0,0,88,88,2,2,2,10,2,2,금천구
22927,22927,24,1,2,0,88,88,0,88,88,7,0,20,1,4,2,7,2,1,금천구
22928,22928,56,2,3,0,88,88,0,88,88,3,2,0,1,4,2,7,1,1,금천구


In [3]:
data_pbhlth_code = pd.read_excel('../raw/지역사회건강조사 2019년 원시자료 이용지침서 등/지역사회건강조사 2019 원시자료 참고사항_보건소정보, 세대유형.xlsx',header=3)
data_pbhlth_code = data_pbhlth_code[data_pbhlth_code['시도코드'] == 11][['보건소코드','보건소명']]
data_pbhlth_code["보건소명"] = data_pbhlth_code["보건소명"].str.replace('보건소', '', regex=True)
data_pbhlth_code.columns=['보건소코드','자치구']
list_region = list(data_pbhlth_code.to_dict()['자치구'].values()) + ['서울시']
list_region.reverse()

In [4]:
df = data_raw[['만나이','성별','자치구', '중등도 신체활동 일수', '중등도 신체활동 시간(시)','중등도 신체활동 시간(분)']]

In [5]:
df1=df[df['중등도 신체활동 일수'] != 99]
df2=df1[df1['중등도 신체활동 일수'] != 77]

## 격렬한 운동 관련
# 시간 --> 분으로 환산
def transtime(df_activity) :
    if 0 <= df_activity['격렬한 신체활동 시간(시)'] <= 24 and df_activity['격렬한 신체활동 시간(분)'] <= 59 :
        return df_activity['격렬한 신체활동 시간(시)']*60 + df_activity['격렬한 신체활동 시간(분)']
    elif df_activity['격렬한 신체활동 시간(시)'] > 24 and df_activity['격렬한 신체활동 시간(분)'] <= 59 :
        return df_activity['격렬한 신체활동 시간(분)']
    else :
        return 0

df_activity["격렬_시분_환산"] = df_activity.apply(transtime, axis =1)
print("격렬한 운동 시분_환산 합: ",df_activity["격렬_시분_환산"].sum())

# 지표 환산
def health_index(df_activity) :
    if df_activity['격렬한 신체활동 일수'] < 3 :
        return 0
    elif 3 <= df_activity['격렬한 신체활동 일수'] and df_activity['격렬_시분_환산'] >= 20 :
        return 1
    else :
        return 0

df_activity['격렬한 신체활동 실천'] = df_activity.apply(health_index, axis =1)
print('격렬한 신체활동 실천 해당자 합 : ', df_activity['격렬한 신체활동 실천'].sum())
    
# 응답자수(결측치 제외)
def valid_response(df_activity) :
    if df_activity['격렬한 신체활동 일수'] in  [77, 99] :
        return 0
    elif df_activity['격렬한 신체활동 시간(시)'] in [77, 99] :
        return 0
    elif df_activity['격렬한 신체활동 시간(분)'] in [77, 99] :
        return 0
    else :
        return 1
    
df_activity['격렬한 신체활동 응답'] = df_activity.apply(valid_response, axis =1)
print('격렬한 신체활동 응답자 합 : ', df_activity['격렬한 신체활동 응답'].sum())

In [8]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

def transtime(df_activity) :
    if 0 <= df_activity['중등도 신체활동 시간(시)'] <= 24 and df_activity['중등도 신체활동 시간(분)'] <= 59 :
        return df_activity['중등도 신체활동 시간(시)']*60 + df_activity['중등도 신체활동 시간(분)']
    elif df_activity['중등도 신체활동 시간(시)'] > 24 and df_activity['중등도 신체활동 시간(분)'] <= 59 :
        return df_activity['중등도 신체활동 시간(분)']
    else :
        return 0
    
def health_index(df_activity) :
    if df_activity['중등도 신체활동 일수'] < 5 :
        return 0
    elif 5 <= df_activity['중등도 신체활동 일수'] and df_activity['중등도 시분 환산'] >= 30 :
        return 1
    else :
        return 0


df_activity = df2.copy() # 오류제거
df_activity["중등도 시분 환산"] = df_activity.apply(transtime, axis =1)
df_activity['중등도 신체활동 실천'] = df_activity.apply(health_index, axis =1)
df_activity = df_activity.drop(['중등도 신체활동 시간(시)','중등도 신체활동 시간(분)','중등도 신체활동 일수','중등도 시분 환산'],axis=1)

In [9]:
list_df = []
for value in list_region:
    temp = df_activity['자치구'] == value
    list_df.append(df_activity[temp])
list_df[0] = df_activity

In [10]:
list_age = ['19~200','19~44','45~64','65~74','75~200']
list_sex = ['계','남','여']
idx_sex = []
idx_age = []
idx_region = []
value_act = []
for i, value1 in enumerate(list_region):
    for j in list_age:
        for k in list_sex:            
            temp = list_df[i][list_df[i]['만나이']>=int(j.split('~')[0])]
            temp1 = temp[temp['만나이'] < int(j.split('~')[1])]
            if k == '남':
                temp2 = temp1[temp1['성별'] == 1]
            elif k == '여':
                temp2 = temp1[temp1['성별'] == 2]
            else:
                temp2 = temp1
            temp3 = temp2['중등도 신체활동 실천'].value_counts(normalize = True).to_dict()
            if 1 not in temp3:
                temp3[1] = 0
            if 0 not in temp3:
                temp3[0] = 0
            value_act.append(round(temp3[1]*100,1))
            
            if j == '19~200':
                idx_age.append('계')
            elif j == '75~200':
                idx_age.append('75 이상')
            else:
                idx_age.append(j)
            idx_sex.append(k)
            idx_region.append(value1)

In [11]:
df1 = pd.DataFrame({'중등도 신체활동 실천':value_act, '지역':idx_region,'연령별':idx_age,'성별':idx_sex})

df1 = df1.set_index(['지역','연령별','성별'])

df1

중등도 신체활동 실천
지역   연령별   성별             
서울시  계     계          10.4
           남          12.3
           여           8.9
     19~44 계           8.8
           남          11.4
           여           6.5
     45~64 계          11.9
           남          12.7
           여          11.4
     65~74 계          12.7
           남          14.2
           여          11.6
     75 이상 계           7.8
           남          11.5
           여           5.0
영등포구 계     계          11.0
           남          13.1
           여           9.1
     19~44 계          10.8
           남          14.9
           여           6.9
     45~64 계          10.5
           남           9.8
           여          11.1
     65~74 계          11.5
           남          12.5
           여          10.7
     75 이상 계          14.5
           남          18.9
           여           8.0
양천구  계     계          10.4
           남          11.6
           여           9.4
     19~44 계           9.1
           남          12.6
           여           6.2
     45~64 계          10.6
           남          11.8
           여           9.8
     65~74 계          18.5
           남          16.7
           여          20.0
     75 이상 계           7.0
           남           4.0
           여           8.7
마포구  계     계           7.8
           남           8.5
           여           7.2
     19~44 계           5.9
           남           6.5
           여           5.5
     45~64 계           6.3
           남           8.0
           여           5.1
     65~74 계          11.5
           남          10.4
           여          12.3
     75 이상 계          14.4
           남          14.9
           여          14.0
동작구  계     계           9.0
           남          12.1
           여           6.5
     19~44 계           6.5
           남           9.0
           여           4.2
     45~64 계          12.0
           남          14.3
           여          10.3
     65~74 계          11.3
           남          15.4
           여           7.9
     75 이상 계           9.3
           남          16.2
           여           5.0
금천구  계     계          12.6
           남          16.4
           여           9.3
     19~44 계          10.6
           남          13.5
           여           7.9
     45~64 계          14.5
           남          18.6
           여          11.3
     65~74 계          16.2
           남          21.7
           여          10.5
     75 이상 계           9.1
           남          14.6
           여           4.3
구로구  계     계           6.8
           남           7.9
           여           5.9
     19~44 계           3.4
           남           3.0
           여           3.7
     45~64 계          10.9
           남          13.2
           여           8.9
     65~74 계           7.2
           남           8.3
           여           6.5
     75 이상 계           1.1
           남           2.2
           여           0.0
관악구  계     계           8.4
           남          11.3
           여           6.2
     19~44 계           8.7
           남          14.2
           여           3.6
     45~64 계           7.5
           남           7.2
           여           7.6
     65~74 계           9.6
           남          10.9
           여           9.0
     75 이상 계           6.3
           남           8.5
           여           4.2
강서구  계     계           9.9
           남          10.1
           여           9.7
     19~44 계           7.5
           남           9.5
           여           5.6
     45~64 계          13.8
           남          12.0
           여          15.0
     65~74 계           7.1
           남           8.3
           여           6.2
     75 이상 계           6.0
           남           2.6
           여           8.7
중랑구  계     계          11.4
           남          14.5
           여           9.3
     19~44 계          10.8
           남          14.8
           여           8.0
     45~64 계          13.0
           남          12.7
           여          13.2
     65~74 계          13.3
           남          17.3
           여       